In [1]:
from pathlib import Path
import traceback
import pandas as pd
from tqdm import tqdm

In [2]:
from Bio.SVDSuperimposer import SVDSuperimposer
from Bio.PDB import PDBParser
import numpy as np

atoms_list = {
    "backbone": ["N", "CA", "C", "O"],
}



def get_paratope_rmsds(pdb_code: str, model_name: str, benchmark_folder: Path):
    parser = PDBParser()
    native = benchmark_folder/f'{pdb_code}/{pdb_code}_true_complex.pdb'
    if 'ensemble' in model_name:
        model_type = model_name.split('_')[0]
        rank = model_name.split("_")[-1]
        if 'ABB2' in model_name:
            model = benchmark_folder/f'{pdb_code}/{model_type}_ensemble_models_{pdb_code}/{rank}_refined.pdb'
        else:
            model = benchmark_folder/f'{pdb_code}/{model_type}_ensemble_models_{pdb_code}/{pdb_code}_rank_{rank}.pdb'
    else:
        model = benchmark_folder/f'{pdb_code}/{model_name}_{pdb_code}_antibody_model_imgt.pdb'

    model_chains = {chain.id: chain for chain in parser.get_structure('model', model).get_chains()}
    native_chains = {chain.id: chain for chain in parser.get_structure('native', native).get_chains()
                     if chain.id in model_chains.keys()}

    if set(native_chains.keys()) != set(model_chains.keys()):
        raise ValueError("Model chain ids not equal to native chain ids.")

    paratope_def_nums = {chain_id: set() for chain_id in model_chains.keys()}

    with open(benchmark_folder/f'{pdb_code}/{pdb_code}_constraint_pairs.txt') as file:
        file.readline()
        for line in file:
            antibody_line = line.split(':')[0]
            words = antibody_line.strip().split(',')
            if words[1][-1].isalpha():
                insert_code = words[1][-1]
                number = int(words[1][:-1])
            else:
                insert_code = ' '
                number = int(words[1])
            paratope_def_nums[words[0]].add((' ', number, insert_code))
            model_resname =  model_chains[words[0]][(' ', number, insert_code)].resname
            if model_resname != words[2].upper():
                raise ValueError(f"For {pdb_code=}, chain id {words[0]} residue {number}{insert_code}, "
                                 f"got mismatching residue to constraint, {model_resname=}, constraint={words[2]}")

    try:
        native_paratope_atom_coords = [list(atom.coord)  for chain_id, residue_keys
                                       in paratope_def_nums.items() for residue_key in residue_keys
                                       for atom in native_chains[chain_id][residue_key]
                                       if atom.get_id() in atoms_list['backbone']]

        model_paratope_atom_coords = [list(atom.coord)  for chain_id, residue_keys
                                       in paratope_def_nums.items() for residue_key in residue_keys
                                       for atom in model_chains[chain_id][residue_key]
                                       if atom.get_id() in atoms_list['backbone']]
    except Exception as e:
        print(traceback.format_exc())
        raise ValueError(f"For {pdb_code=}, {model_name=}, got error {e}")


    svd = SVDSuperimposer()
    svd.set(np.array(native_paratope_atom_coords), np.array(model_paratope_atom_coords))
    svd.run()
    rmsd_para = svd.get_rms()
    row = {'pdb': pdb_code, 'model': model_name, 'rmsd_paratope': rmsd_para}

    return row

In [3]:
records = []
model_names = (["ABodyBuilder2", "ABlooper", "AF2", "IgFold"]+ [f"ABB2_ensemble_rank{i}" for i in range(4)]
               + [f"IgFold_ensemble_rank_{i}" for i in range(4)] + [f"AF2_ensemble_rank_{i}" for i in range(5)]
               )
benchmark_folder = Path('../../benchmark_haddock_27_July_2024')
for path in tqdm(list(benchmark_folder.iterdir())):
    if path.is_dir():
        pdb_code = path.name
    else:
        continue
    print(pdb_code)
    for model_name in model_names:
        try:
            records.append(get_paratope_rmsds(pdb_code,model_name,benchmark_folder))
        except Exception as e:
            print(f"Got error {e} for {pdb_code=}, {model_name=}.")

  0%|          | 0/84 [00:00<?, ?it/s]

7rfb_A0-B0


  1%|          | 1/84 [00:02<03:05,  2.23s/it]

7ps1_A0-B0


  2%|▏         | 2/84 [00:03<02:13,  1.62s/it]

...

 98%|█████████▊| 82/84 [01:46<00:02,  1.23s/it]

7rks_H0-L0


 99%|█████████▉| 83/84 [01:48<00:01,  1.22s/it]

7kf0_H0-L0


100%|██████████| 84/84 [01:49<00:00,  1.30s/it]


In [4]:
paratope_rmsd_df = pd.DataFrame.from_records(records)

In [7]:
paratope_rmsd_df.to_csv('../data/paratope_rmsds.csv')

In [6]:
print(paratope_rmsd_df)

             pdb                model  rmsd_paratope
0     7rfb_A0-B0        ABodyBuilder2       3.427635
1     7rfb_A0-B0             ABlooper       3.984237
2     7rfb_A0-B0                  AF2       3.929652
3     7rfb_A0-B0               IgFold       2.799389
4     7rfb_A0-B0  ABB2_ensemble_rank0       3.427635
...          ...                  ...            ...
1406  7kf0_H0-L0  AF2_ensemble_rank_0       2.011022
1407  7kf0_H0-L0  AF2_ensemble_rank_1       1.873006
1408  7kf0_H0-L0  AF2_ensemble_rank_2       2.009312
1409  7kf0_H0-L0  AF2_ensemble_rank_3       2.049127
1410  7kf0_H0-L0  AF2_ensemble_rank_4       1.850511

[1411 rows x 3 columns]
